In [6]:
import pandas as pd
import numpy as np

time_precision = 4   # jagame päeva xh suurusteks plokkideks


X_train = pd.read_csv("../../data/X_train.csv")
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.ToimAeg = X_train.ToimAeg.values.astype('datetime64[D]')
X_train.ToimAeg = X_train['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_train["Aasta"] = X_train['ToimAeg'].dt.year
X_train["Kuu"] = X_train['ToimAeg'].dt.month
X_train["Päev"] = X_train['ToimAeg'].dt.day
X_train["Tund"] = X_train['ToimAeg'].dt.hour
X_train.drop(columns="ToimAeg", inplace=True)
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.head()

y_train = pd.read_csv("../../data/y_train.csv")
y_train = y_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_train.head()

X_valid = pd.read_csv("../../data/X_valid.csv")
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.ToimAeg = X_valid.ToimAeg.values.astype('datetime64[D]')
X_valid.ToimAeg = X_valid['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_valid["Aasta"] = X_valid['ToimAeg'].dt.year
X_valid["Kuu"] = X_valid['ToimAeg'].dt.month
X_valid["Päev"] = X_valid['ToimAeg'].dt.day
X_valid["Tund"] = X_valid['ToimAeg'].dt.hour
X_valid.drop(columns="ToimAeg", inplace=True)
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.head()

y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.head()

X_train.head()

,Lest_X,Lest_Y,KOHT_APTEEK,KOHT_AVALIK_KOHT,KOHT_BUROO_KONTOR,KOHT_BUSSIJAAM_BUSSITERMINAAL,KOHT_BUSSIPEATUS,KOHT_EHITUS,KOHT_HOTELL_MOTELL,KOHT_KASIINO,...,ToimNadalapaev_Kolmapäev,ToimNadalapaev_Laupäev,ToimNadalapaev_Neljapäev,ToimNadalapaev_Pühapäev,ToimNadalapaev_Reede,ToimNadalapaev_Teisipäev,Aasta,Kuu,Päev,Tund
0,6584000,540500,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,2017,2,5,0
1,6593000,557000,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,2019,6,6,0
2,6461000,663000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2012,11,19,0
3,6590000,539000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,2019,11,11,0
4,6590500,549500,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,2015,3,24,0


In [7]:
y_train.info()
y_train.drop(columns=["Kahjusumma","Koht"], inplace=True)
y_valid.drop(columns=["Kahjusumma", "Koht"], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62476 entries, 0 to 62475
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LIIK_AVALIKU_KORRA_RIKKUMINE   62476 non-null  int8  
 1   LIIK_JALGRATTA_MOPEEDI_VARGUS  62476 non-null  int8  
 2   LIIK_KEHALINE_VAARKOHTLEMINE   62476 non-null  int8  
 3   LIIK_KELMUS                    62476 non-null  int8  
 4   LIIK_KONFLIKT/OMAVOLI          62476 non-null  int8  
 5   LIIK_KOOLIVAGIVALD             62476 non-null  int8  
 6   LIIK_LIIKLUSRIKKUMINE          62476 non-null  int8  
 7   LIIK_MOOTORSOIDUKI_VARGUS      62476 non-null  int8  
 8   LIIK_MUU                       62476 non-null  int8  
 9   LIIK_PISIVARGUS                62476 non-null  int8  
 10  LIIK_ROOVIMINE                 62476 non-null  int8  
 11  LIIK_TULEKAHJU                 62476 non-null  int8  
 12  LIIK_VANDALISM                 62476 non-null  int8  
 13  L

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as MSE

rfc = RandomForestClassifier(n_estimators=100, random_state=1).fit(X_train, y_train)
probs_rfc = rfc.predict_proba(X_valid)

In [ ]:
df_proba = pd.DataFrame(np.array([x.transpose()[1] for x in probs_rfc]).transpose(), columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud


#lihtsalt predict jaoks df_proba = pd.DataFrame(probs_rfc, columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud

#.shape
df_proba
print(MSE(y_valid, df_proba))

#probs_rfc.transpose().shape

In [ ]:
from sklearn import metrics
print(metrics.roc_auc_score(y_valid, df_proba))
for i in df_proba.columns:
    #print(y_valid[i])
    print(metrics.accuracy_score(y_valid[i], df_proba[i]), i, sep="\t")

In [ ]:
for i in df_proba.columns:
    print(sum(df_proba[i]), sum(y_valid[i]), i, sep="\t")

In [ ]:
metrics.accuracy_score(y_valid, df_proba)

In [ ]:
y_valid.head()